In [ ]:
# |default_exp neural

In [ ]:
# |export
import logging
from collections import OrderedDict

import numpy as np
import timm
import torch
from fastai.layers import ConvLayer, LinBnDrop, ResizeBatch, SigmoidRange, Identity
from fastai.tabular.model import TabularModel
from fastai.vision.models.xresnet import *
from fastcore.basics import store_attr
from fastcore.foundation import L
from torch import nn
from torch.distributions.categorical import Categorical
from torch.distributions.normal import Normal

# Neural
> Neural networks models used by the AI to learn

In [ ]:
# |export
def init_weights(m):
    if isinstance(m, (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear)): nn.init.kaiming_normal_(m.weight)

class StateCatNet(nn.Module):
    def __init__(self, **module_mapping):
        super().__init__()
        self.output_dim = 0
        for name, module in OrderedDict(sorted(module_mapping.items())).items():
            if not isinstance(module, nn.Module):
                module = module()
            self.add_module(name, module)
            self.output_dim += module.output_dim

    def forward(self, state):
        if isinstance(state, dict):
            state = [state]
        outputs = [
            module(torch.stack([s[name] for s in state]))
            for name, module in self.named_children()
        ]
        return torch.cat(
            outputs,
            dim=1,
        )


class Flatten(nn.Flatten):
    def __init__(self, input_dim):
        super().__init__()
        self.output_dim = np.prod(input_dim).item()


def truncated_timm_model(name, nb_trunc=1, pretrained=True):
    model = timm.create_model(name, pretrained=pretrained)
    return nn.Sequential(*list(model.children())[:-nb_trunc])

class TimmModel(nn.Module):
    def __init__(self, name, input_dim, positional=False, pretrained=True, global_pool="catavgmax"):
        super().__init__()
        c, h, w = input_dim
        self.positional = positional
        if self.positional:
            c += 2
            vertical = torch.tile(torch.linspace(0,1,h).unsqueeze(1), (1,w))
            horizontal = torch.tile(torch.linspace(0,1,w).unsqueeze(0), (h,1))
            positional_channels = torch.stack((vertical,horizontal)).unsqueeze(0)
            self.register_buffer("positional_channels", positional_channels)
        self.raw_model = timm.create_model(
            name,
            in_chans=c,
            pretrained=pretrained,
            num_classes=0,
            global_pool=global_pool
        )
        self.output_dim = self.raw_model(torch.rand(1, c, h, w)).shape[1]

    def forward(self, x):
        if self.positional:
            x = torch.cat((
                torch.tile(self.positional_channels, (x.shape[0], 1, 1, 1)),
                x
            ), dim=1)
        return self.raw_model(x)
        

class VisualNet(nn.Sequential):
    def __init__(self, model, input_dim, hidden=16):
        if not isinstance(model, nn.Module):
            model = model()
        t = model(torch.rand(1, *input_dim))
        _, c, h, w = t.shape
        resize = ResizeBatch(c*h, w)
        conv1 = ConvLayer(c*h, hidden, ks=1, ndim=1)
        conv2 = ConvLayer(hidden, hidden, ks=5, stride=3, ndim=1)
        flat = nn.Flatten()
        self.output_dim = flat(conv2(conv1(resize(t)))).shape[1]
        super().__init__(model,resize,conv1,conv2,flat)


class FlatLinearNet(nn.Sequential):
    def __init__(self, input_dim, hidden=64, output_dim=32, p=0., act=nn.ReLU(inplace=True)):
        self.output_dim = output_dim
        layers_dim = L(input_dim).product() + L(hidden) + output_dim
        layers = L(nn.Flatten())
        for inp, out in zip(layers_dim, layers_dim[1:-1]):
            layers.append(LinBnDrop(inp, out, bn=False, p=p, act=act))
        layers.append(LinBnDrop(layers_dim[-2], layers_dim[-1], bn=True, p=0.))
        super().__init__(*layers)

class Conv1dFlat(nn.Sequential):
    def __init__(self, input_dim, hidden=16):
        flatten = Flatten(L(hidden) + input_dim[1:])
        self.output_dim = flatten.output_dim
        super().__init__(nn.Conv1d(input_dim[0], hidden, 1), nn.ReLU(), flatten)


class DuelingNet(nn.Module):
    def __init__(self, model, output_dim, hidden=256, p=0.0):
        super().__init__()
        store_attr()
        if not isinstance(self.model, nn.Module):
            self.model = self.model()
        self.value_stream = FlatLinearNet(self.model.output_dim, hidden, 1, p)
        self.advantage_stream = FlatLinearNet(self.model.output_dim, hidden, output_dim, p)

    def forward(self, state):
        features = self.model(state)
        values = self.value_stream(features)
        advantages = self.advantage_stream(features)
        return values + (advantages - advantages.mean())

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()